*Enter your observations here*

In [1]:
import mailbox
import csv
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import re
import pandas as pd
import numpy as np
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA





stop_words = set(stopwords.words('english'))
print(stop_words)

def createCSV(name, mbox, mes_type):
    #createing SPAM csv
    writer = csv.writer(open(name, "w", encoding="utf-8"))
    writer.writerow(['Subject', 'From', 'Date', 'Message-ID', 'Payload', 'Type'])
    for message in mailbox.mbox(mbox):

        if message.is_multipart():
            content = ''.join(str(part.get_payload(decode=True)) for part in message.get_payload())
            content = cleanMe(content)
        else:
            content = str(message.get_payload(decode=True))
            content = cleanMe(content)
        content = content.split()
        print(content)
        writer.writerow([message['subject'], message['from'], message['date'], message['Message-Id'], content, mes_type])
    

{'of', 'here', 'no', 'don', 'needn', "mustn't", 'if', 'out', 'in', 'our', "you'd", 'themselves', 'y', 'aren', 'any', 'into', "hasn't", "needn't", 'are', 'nor', 'couldn', "it's", 'below', 'be', 'didn', 'ain', 'these', 'should', "shan't", 'whom', 'do', 'other', 'you', 'some', "mightn't", 'through', 'off', 'from', 'herself', 'haven', 'll', 're', 'being', 'where', "aren't", 'not', "don't", 'himself', 'what', 'an', 'while', 'why', 'few', 'a', 'm', 'was', 'at', 'as', 'wasn', 'once', 'it', 'they', 'only', 'is', "she's", 'until', 'did', 't', 'between', 'on', 'yours', 'too', 'both', 'having', 'against', 'those', "you've", 'doing', "haven't", "you're", 'theirs', "won't", 'down', "wouldn't", 'his', 'before', 'above', 'further', 'her', 'd', "isn't", 'wouldn', 'she', 'hadn', 'but', 'during', 'ourselves', 'then', 'who', "shouldn't", 'after', 'hasn', 'own', "wasn't", 'mightn', 'were', 'under', 'been', 'so', 'more', "you'll", 've', 'with', 'most', 'which', "weren't", 'myself', 'itself', 'up', 'isn', '

In [2]:
def cleanMe(html):
    soup = bs(html,"lxml") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n '.join(chunk for chunk in chunks if chunk)
    text = text.replace('\\n', ' ').replace('\\t', ' ').replace("\n", ' ').replace('\\b', ' ')
    # gets rid of escape characters
    text = bytes(text, "utf-8").decode("unicode_escape")
    # gets rid of links 
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    text = " ".join(re.findall(r'\s+[a-zA-Z]+\s+', text))
    # make text lower case
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    # remove stop words before csv was written
    text = word_tokenize(text)
    resultwords  = [word for word in text if word not in stop_words]
    result = ' '.join(resultwords)
    return result




In [3]:
#createCSV('./data/spam_t1.csv', './data/Spam.mbox', 'Spam')
#createCSV('./data/inbox_t1.csv', './data/Inbox.mbox', 'Ham')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

#spam1 = pd.read_csv("./data/spam1.csv")
spam2 = pd.read_csv("./data/spam_t1.csv")
#print(spam1.shape)
#inbox1 = pd.read_csv("./data/inbox.csv")
inbox2 = pd.read_csv("./data/inbox_t1.csv")
#print(inbox2.shape[0])
#spam2.replace('Spam', 0).replace('Ham', 1)
#spam2.data = np.matrix(spam2.Payload)
print(spam2.shape)
print(inbox2.shape)
# using sklearn package to make prior matrix for ham and spam
# inbox is too big, delete some rows (look in past year)
inbox2_current = inbox2[0 : 1000]
print(inbox2_current.shape)
vec = TfidfVectorizer()
def getPrior(data, result):
    X = vec.fit_transform(data)
    prior = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
    prior['RESULT'] = result
    return prior

spam_prior = getPrior(spam2.Payload, 0)

ham_prior = getPrior(inbox2_current.Payload,1)
ham_prior
spam_prior

(707, 6)
(16238, 6)
(1000, 6)


,aaron,abandoned,abandoning,abby,abib,abilities,ability,able,abonelik,abonelikler,...,yuan,za,zeldes,zenith,zillow,zinc,zippyloan,zone,zvolit,RESULT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.377973,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0


In [5]:

frames = [spam_prior, ham_prior]
# replace NaN with zero because it means that spam did not have that word, or spam did not have that word
prior = pd.concat(frames).fillna(0)


In [6]:
# breaking up prior (propabilities of word, ham/spam, 0 == spam, 1 == spam)
target = prior.RESULT
data = prior.drop(["RESULT"], axis = 1)


In [7]:
# inbox concatenated vectors = icv
# icv_id = []
# icv_word = []
# icv_target = []

# spam_start = time.time()
# for index, row in spam1.iterrows():
#     email_words = str(row['Payload']).strip()
#     unique_id = row["Message-ID"]
#     words_dict = {}
#     words_vec = email_words.split(" ")
#     for word in words_vec:
#         words_dict[word] = True
#     for word in words_dict.keys():
#         icv_id.append(unique_id)
#         icv_word.append(word)
#         icv_target.append(1)

# print("Spam Time taken: ", str(time.time() - spam_start))

# inbox_start = time.time()
# for index, row in inbox1.iterrows():
#     email_words = str(row['Payload']).strip()
#     unique_id = row["Message-ID"]
#     words_dict = {}
#     words_vec = email_words.split(" ")
#     for word in words_vec:
#         words_dict[word] = True
#     for word in words_dict.keys():
#         icv_id.append(unique_id)
#         icv_word.append(word)
#         icv_target.append(0)

# print("Ham Time taken: ", str(time.time() - inbox_start))
            
# new_df = pd.DataFrame({"id":icv_id, "word": icv_word, "target": icv_target})


In [8]:
# #word_spam_df = new_df
# #word_spam_df_features = word_spam_df.loc[:, "word"]

# word_spam_df_outcomes = word_spam_df.loc[:, "target"]
# word_spam_df.shape
# word_spam_df
# # tyring to take out stop words after df is made. didn't do 
# df = word_spam_df[word_spam_df.word != stop_words]
# df.shape

In [9]:
# total_ham = inbox1.shape[0]
# total_spam = spam1.shape[0]
# total = total_spam + total_ham
# p_ham = total_spam / total
# p_spam = total_ham / total
# print(p_ham, p_spam)


In [10]:
# def getUniqueWordCounts(word_and_spam, ham_total, spam_total, ham_p, spam_p):
#     seen_words = {}
#     word_probs = []
    
#     unique_words = word_and_spam.word.unique()
#     unique_words_for_priors = []
#     unique_word_counts = {}
#     for w in unique_words:
#         unique_word_counts[w] = {"s":0, "h":0}
       
#     for index, row in word_and_spam.iterrows():
#         word = row['word']
#         if (word not in seen_words):
#             if (row["target"] == 1):
#                 unique_word_counts[word]["s"] = unique_word_counts[word]["s"] + 1
#             else:
#                 unique_word_counts[word]["h"] = unique_word_counts[word]["h"] + 1
                
# #             w_r = word_and_spam[word_and_spam["word"] == word]
# #             spam_word_count = len(w_r[(word_and_spam["target"] == 1)])
# #             ham_word_count = len(w_r[(word_and_spam["target"] == 0)])
# #             p_spam_word = spam_word_count / spam_total
# #             p_ham_word = ham_word_count / ham_total
# #             p_s = p_spam_word * spam_p
# #             p = (p_s)  / (p_s + p_ham_word * ham_p)
# #             word_probs.append({"key": word, "val": p})
            
# #     spam_status_for_unique_words = []
# #     for w in unique_words:
# #         spam_status_for_unique_words.append(0)
# #         spam_status_for_unique_words.append(1)
# #         unique_words_for_priors.append(w)
# #         unique_words_for_priors.append(w)
        
#     # these could also perhaps be total spam or ham emails?
# #     for index, row in word_and_spam.iterrows():
# #         word = row['word']
# #         if (word not in seen_words):
# #             seen_words[word] = True
# #             w_r = word_and_spam[word_and_spam["word"] == word]
# #             spam_word_count = len(w_r[(word_and_spam["target"] == 1)])
# #             ham_word_count = len(w_r[(word_and_spam["target"] == 0)])
# #             p_spam_word = spam_word_count / spam_total
# #             p_ham_word = ham_word_count / ham_total
# #             p_s = p_spam_word * spam_p
# #             p = (p_s)  / (p_s + p_ham_word * ham_p)
# #             word_probs.append({"key": word, "val": p})
#     return unique_word_counts


# counts_df = getUniqueWordCounts(word_spam_df, total_ham, total_spam, p_ham, p_spam)

In [11]:
# counts_df_not_dict = pd.DataFrame(counts_df)
# total_ham = inbox1.shape[0]
# total_spam = spam1.shape[0]
# total = total_spam + total_ham
# p_ham = total_spam / total
# p_spam = total_ham / total
# print(p_ham, p_spam)

# word_arr = []
# target_arr = []
# p_arr = []
# for key, val in counts_df.items():
#     spam_word_count = val["s"]
#     ham_word_count = val["h"]
    
#     p_word_given_spam = spam_word_count / total_spam
#     p_word_given_ham = ham_word_count / total_ham    
#     p_s = p_word_given_spam * p_spam
#     p_h = p_word_given_ham * p_ham
    
#     p_spam_given_word = (p_s)  / (p_s + p_h)
#     p_ham_given_word = (p_h)  / (p_s + p_h)
    
#     word_arr.append(key)
#     target_arr.append(1)
#     p_arr.append(p_spam_given_word)
#     word_arr.append(key)
#     target_arr.append(0)
#     p_arr.append(p_ham_given_word)

    
# final_df = pd.DataFrame(data={"word":word_arr,"target":target_arr,"p":p_arr})

In [12]:
# final_df.to_csv("priors", sep=",")


In [13]:
# counts_df_not_dict

In [14]:
# didn't get time to write this...but I think the getProirs dictionary could get changed to a matrix easily
#matrixPrior(words_spam):
#    for 

In [15]:
from sklearn import naive_bayes as nb
from sklearn.model_selection import train_test_split

# seperating feature (prior) matrix
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33)


In [16]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix 
from sklearn.ensemble import RandomForestClassifier

model1 = MultinomialNB()
model2 = LinearSVC()
model3 = RandomForestClassifier()

model1.fit(X_train,y_train)
model2.fit(X_train, y_train)
# for extra credit :)
model3.fit(X_train, y_train)

result1 = model1.predict(X_test)
result2 = model2.predict(X_test)
result3 = model3.predict(X_test)

# cheking results without cross validation

print("Confusion Matrix for MultinomialNB:\n", confusion_matrix(y_test,result1))
print("Confusion Matrix for SVC:\n", confusion_matrix(y_test,result2))
print("Confusion Matrix for RandomForest:\n", confusion_matrix(y_test, result3))


Confusion Matrix for MultinomialNB:
 [[127 116]
 [  6 315]]
Confusion Matrix for SVC:
 [[222  21]
 [ 28 293]]
Confusion Matrix for RandomForest:
 [[222  21]
 [ 16 305]]


In [37]:
# parameters for pipeline

# random forest
pg_3 = {'randomforestclassifier__n_estimators': np.arange(1,10)}

# multinomial Niave Bayse
pg_1 = {'multinomialnb__alpha': np.arange(.5,1.5),
        'multinomialnb__fit_prior': [True, False]}

# svc classifier
pg_2 = {'linearsvc__dual': [True, False],
        'linearsvc__fit_intercept': [True, False]}




In [33]:
def run_model(model, param_grid, xtrain, ytrain, do_pca = False):
    scaler = MinMaxScaler()
    if(do_pca == True):
        pca = PCA(n_components = 10)
        pipe = make_pipeline(pca, model)
    else:
        pipe = make_pipeline(model)
    grid = GridSearchCV(pipe,param_grid)
    grid.fit(xtrain, ytrain)
    grid.best_params_
    accuracy = grid.score(xtrain, ytrain)
    print(f"In-sample accuracy: {accuracy:0.2%}", model)
    return(grid)

In [34]:
grid3 = run_model(model3, pg_3, X_train, y_train)
grid1 = run_model(model1, pg_1, X_train, y_train)
grid2 = run_model(model2, pg_2, X_train, y_train)


In-sample accuracy: 98.86% RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
In-sample accuracy: 95.63% MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
In-sample accuracy: 98.86% LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


In [36]:
preds3 = grid3.predict(X_test)
preds2 = grid2.predict(X_test)
preds1 = grid1.predict(X_test)
matrix3 = confusion_matrix(y_test, preds3)
print(matrix3)
matrix2 = confusion_matrix(y_test, preds2)
print(matrix2)
matrix1 = confusion_matrix(y_test, preds1)
print(matrix1)

[[219  24]
 [ 21 300]]
[[222  21]
 [ 28 293]]
[[204  39]
 [ 32 289]]
